In [1]:
from importlib import reload
import json
import pretty_midi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# check gpu
print(tf..device_lib.list_local_devices())

In [33]:
import modules.midiMethods as midiMethods
reload(midiMethods)
from modules.midiMethods import *

import modules.dataMethods as dataMethods
reload(dataMethods)
from modules.dataMethods import *

import modules.models as models
reload(models)

import modules.mlClasses as mlClasses
reload(mlClasses)


<module 'modules.mlClasses' from '/home/ubuntu/music-ml/380-music-representation/modules/mlClasses.py'>

In [39]:
# if needed, read in filenames... specify if validation or test
maestro = pd.read_csv('training_data/maestro-v2.0.0withPeriod.csv', index_col=0)
filenames = list(maestro[maestro['split'] == 'test']['midi_filename'])

In [ ]:
# or process data set
data_path = 'training_data/MaestroV2.00/maestro-v2.0.0/'
test, exceeded = files2note_bin_examples(data_path, filenames, skip=1, n_notes=256, speed=1)
print(len(examples))

######## Example no. 1 of 178 , length 721
notes:  514
total_durations_exceeded:  0
total_shifts_exceeded:  0
notes_lost:  209
######## Example no. 2 of 178 , length 783
notes:  1028
total_durations_exceeded:  1
total_shifts_exceeded:  0
notes_lost:  481
######## Example no. 3 of 178 , length 3648
notes:  4626
total_durations_exceeded:  1
total_shifts_exceeded:  0
notes_lost:  545
######## Example no. 4 of 178 , length 3878
notes:  8224
total_durations_exceeded:  1
total_shifts_exceeded:  1
notes_lost:  840
######## Example no. 5 of 178 , length 4060
notes:  11822
total_durations_exceeded:  2
total_shifts_exceeded:  1
notes_lost:  1317
######## Example no. 6 of 178 , length 3633
notes:  15420
total_durations_exceeded:  2
total_shifts_exceeded:  1
notes_lost:  1366
######## Example no. 7 of 178 , length 7732
notes:  22102
total_durations_exceeded:  4
total_shifts_exceeded:  3
notes_lost:  2446
######## Example no. 8 of 178 , length 2884
notes:  24672
total_durations_exceeded:  4
total_s

In [38]:
# store examples in json format
with open('training_data/note_bin_v1/nb_256_val.json', 'w') as f:
    json.dump(val, f)

In [8]:
# or read them in
with open('training_data/training_data_note_bin_complete.json', 'r') as f:
    examples = json.load(f)

In [28]:
note_bin2pm(exceeded[5]).write('midi/exceeded.mid')

In [8]:
# build simple model
# excellent example of recurrent model here https://www.tensorflow.org/tutorials/text/text_generation
hidden_state = 512
lstm_layers = 3
model2 = models.create_model1(hidden_state_size=hidden_state, lstm_layers=lstm_layers)
training_generator = mlClasses.DataGenerator(examples, augment=True)
val_gen = 

Model: "3layerLSTM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 6)]     0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 128, 88)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 10)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 128, 60)      0           input_1[0][0]                    
_________________________________________________________________________________________

NameError: name 'examples' is not defined

In [51]:
model2.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# checkpoint = tf.keras.callbacks.ModelCheckpoint("weights/model1/{epoch:02d}-{train_loss:.2f}.hdf5", monitor='train_loss', verbose=2, save_best_only=True, save_weights_only=True)
epochs=80
history = model2.fit_generator(training_generator, epochs=epochs)
model2.save_weights('weights/note_bin/model2_60epochs512state3layer.h5')
with open(f'models/nbmodel3/history{epochs}e.json', 'w') as f:
    json.dump(history, f)

Epoch 1/50
803/803 [==============================] - 331s 413ms/step - loss: 9.3545 - dense_24_loss: 2.4480 - dense_25_loss: 0.0392 - dense_26_loss: 2.1218 - dense_27_loss: 0.3382 - dense_28_loss: 2.1147 - dense_29_loss: 2.2925 - dense_24_accuracy: 0.3311 - dense_25_accuracy: 0.9877 - dense_26_accuracy: 0.3010 - dense_27_accuracy: 0.8892 - dense_28_accuracy: 0.3347 - dense_29_accuracy: 0.1973
Epoch 2/50
803/803 [==============================] - 327s 407ms/step - loss: 9.2848 - dense_24_loss: 2.4131 - dense_25_loss: 0.0385 - dense_26_loss: 2.1093 - dense_27_loss: 0.3345 - dense_28_loss: 2.1032 - dense_29_loss: 2.2861 - dense_24_accuracy: 0.3395 - dense_25_accuracy: 0.9878 - dense_26_accuracy: 0.3029 - dense_27_accuracy: 0.8903 - dense_28_accuracy: 0.3374 - dense_29_accuracy: 0.1985
Epoch 3/50
803/803 [==============================] - 328s 409ms/step - loss: 9.2235 - dense_24_loss: 2.3822 - dense_25_loss: 0.0379 - dense_26_loss: 2.0985 - dense_27_loss: 0.3313 - dense_28_loss: 2.0932 -

In [45]:
# This works just like this:
import modules.models as models
reload(models)    

<module 'modules.models' from '/home/ubuntu/music-ml/380-music-representation/modules/models.py'>

In [9]:
# tf.train.latest_checkpoint('weights\first_attempt')
prediction_model = models.create_model1(batch_size=1, stateful=True, hidden_state_size=hidden_state, lstm_layers=lstm_layers)

Model: "3layerLSTM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(1, 128, 6)]        0                                            
__________________________________________________________________________________________________
lambda_6 (Lambda)               (1, 128, 88)         0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_7 (Lambda)               (1, 128, 10)         0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_8 (Lambda)               (1, 128, 60)         0           input_2[0][0]                    
_________________________________________________________________________________________

In [10]:
prediction_model.load_weights('weights/note_bin/model2_60epochs512state3layer.h5')
for i in range(1,5):
    temperatures = [0.02 + i * 0.003] * 6
    # temperatures = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
    # input_notes = [[44,0,0,4,3,20], [48,0,0,4,3,20], [51,0,0,3,3,20],[56,0,5,3,3,20]]
    input_notes = [[40,0,3,3,3,10],[42,3,3,3,3,10],[44,3,3,3,3,10],[45,3,3,3,3,10],[40,3,3,3,3,10],[44,0,0,3,3,10],[47,0,0,3,3,10],[52,0,7,3,3,10]]
    new_seq_length = 512
    new_seq = models.generate_music(prediction_model, new_seq_length, temperatures, input_notes)
    ns = [[int(a) for a in note] for note in new_seq]
    pm = note_bin2pm(ns)
    pm.write(f'midi/model512ss3l/60epochs/scale{new_seq_length}{temperatures[0]}.mid')

[40, 0, 3, 3, 3, 10]
[42, 3, 3, 3, 3, 10]
[44, 3, 3, 3, 3, 10]
[45, 3, 3, 3, 3, 10]
[40, 3, 3, 3, 3, 10]
[44, 0, 0, 3, 3, 10]
[47, 0, 0, 3, 3, 10]
[40, 0, 3, 3, 3, 10]
[42, 3, 3, 3, 3, 10]
[44, 3, 3, 3, 3, 10]
[45, 3, 3, 3, 3, 10]
[40, 3, 3, 3, 3, 10]
[44, 0, 0, 3, 3, 10]
[47, 0, 0, 3, 3, 10]
[40, 0, 3, 3, 3, 10]
[42, 3, 3, 3, 3, 10]
[44, 3, 3, 3, 3, 10]
[45, 3, 3, 3, 3, 10]
[40, 3, 3, 3, 3, 10]
[44, 0, 0, 3, 3, 10]
[47, 0, 0, 3, 3, 10]
[40, 0, 3, 3, 3, 10]
[42, 3, 3, 3, 3, 10]
[44, 3, 3, 3, 3, 10]
[45, 3, 3, 3, 3, 10]
[40, 3, 3, 3, 3, 10]
[44, 0, 0, 3, 3, 10]
[47, 0, 0, 3, 3, 10]


In [1]:
models.plt_metric(history)


NameError: name 'models' is not defined

In [24]:
a = np.array([[87, 0, 0, 4, 3, 20], [2, 0, 0, 4, 3, 20]])
semitones = -4
for j in range(2):
    a[j,0] = min(max(a[j,0] + semitones, 0), 87)
print(a)

[[83  0  0  4  3 20]
 [ 0  0  0  4  3 20]]
